<a href="https://colab.research.google.com/github/dakilaledesma/NCBG/blob/main/train/Multilabel_Parts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%time
! unzip -q /content/drive/MyDrive/UNC/NCBG/sept1_classif_training.zip -d /content
! cp /content/drive/MyDrive/UNC/NCBG/qry_fsus_trainingdata_ai.xlsx /content/qry_fsus_trainingdata_ai.xlsx

CPU times: user 53.4 ms, sys: 15.9 ms, total: 69.3 ms
Wall time: 5.85 s


In [2]:
! mv /content/content/images /content/images
! rm -rf /content/content

In [3]:
! pip install -U tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 578.0 MB 13 kB/s 
     |████████████████████████████████| 438 kB 53.5 MB/s 
     |████████████████████████████████| 1.7 MB 49.5 MB/s 
     |████████████████████████████████| 5.9 MB 46.0 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5

In [4]:
! apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following packages will be REMOVED:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be upgraded:
  libcudnn8
1 upgraded, 0 newly installed, 1 to remove and 18 not upgraded.
Need to get 430 MB of archives.
After this operation, 3,139 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  libcudnn8 8.1.0.77-1+cuda11.2 [430 MB]
Fetched 430 MB in 8s (54.1 MB/s)
(Reading database ... 155685 files and directories currently installed.)
Removing libcudnn8-dev (8.0.5.39-1+cuda11.1) ...
(Reading database ... 155663 files and directories currently installed.)
Preparing to unpack .../libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb ...
Unpacking libcudnn8 (8.1.0.77-1+c

In [5]:
! pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.2 MB/s 


In [1]:
import pandas as pd

df = pd.read_excel("qry_fsus_trainingdata_ai.xlsx")

In [2]:
print(df.columns)

Index(['file', 'directory', 'imagequality', 'image_context'], dtype='object')


In [3]:
classes = list(set(sum([v.split(';') for v in df["image_context"] if type(v) == str], [])))
classes.sort()
class_dict = {v: i for i, v in enumerate(classes)}

In [4]:
print(class_dict)

{'aspect': 0, 'bud': 1, 'fall-color': 2, 'flower': 3, 'fruit': 4, 'leaf': 5, 'leaf-underside': 6, 'other': 7, 'root': 8, 'seed': 9, 'side-by-side': 10, 'stem': 11, 'venation': 12, 'vestiture': 13, 'whole-plant': 14}


In [5]:
import numpy as np

im_to_class = {}
for idx, row in df.iterrows():
  anno_bn = row["file"]
  try:
    classes = row["image_context"].split(';')
  except AttributeError:
    continue
  
  out_anno = np.zeros(15, dtype=int)
  for class_ in classes:
    class_idx = class_dict.get(class_)
    out_anno[class_idx] = 1
  
  im_to_class[anno_bn] = out_anno

In [6]:
from glob import glob
import os
from PIL import Image, UnidentifiedImageError
from tqdm.notebook import tqdm

X = []
y = []
for im_fn in glob("images/*.*"):
  bn = os.path.basename(im_fn).split("__")[1].replace(".jpg", '')
  bn = f"{bn}.jpg"
  try:
    y.append(im_to_class[bn])
    im = Image.open(im_fn)
    im = im.resize((331, 331))
    im = np.array(im, dtype=int)
  except KeyError or UnidentifiedImageError:
    continue

  X.append(im)

In [7]:
print(len(X), len(y))

8489 8489


In [8]:
X = np.array(X, dtype=int)
y = np.array(y, dtype=int)

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.convnext import ConvNeXtBase
from tensorflow.keras.layers import Dense

ConvNeXt top layer configured as such

```
    if include_top:
        x = Head(num_classes=classes, name=model_name)(x)
        imagenet_utils.validate_activation(classifier_activation, weights)
    else:
        if pooling == "avg":
            x = layers.GlobalAveragePooling2D()(x)
        elif pooling == "max":
            x = layers.GlobalMaxPooling2D()(x)
        x = layers.LayerNormalization(epsilon=1e-6)(x)
```



In [10]:
backbone = ConvNeXtBase(input_shape=(331, 331, 3), include_top=False, pooling='avg')
model = Sequential()
model.add(backbone)
model.add(Dense(15, activation="sigmoid"))

In [11]:
print(X.shape, y.shape)

(8489, 331, 331, 3) (8489, 15)


In [12]:
gen = tf.keras.preprocessing.image.ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=10,
    shear_range=10.0,
    zoom_range=0.2
)
res = gen.flow(x=X, y=y, batch_size=8)
x1, y1 = next(res)

In [ ]:
import tensorflow_addons as tfa

opt = tfa.optimizers.AdaBelief(learning_rate=1e-3)
model.compile(optimizer=opt, loss="binary_crossentropy", metrics="categorical_accuracy")
model.fit(res, epochs=80)

/usr/local/lib/python3.7/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.7.0 and strictly below 2.10.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


Epoch 1/80
1062/1062 [==============================] - 807s 652ms/step - loss: 0.1903 - categorical_accuracy: 0.5730
Epoch 2/80
1062/1062 [==============================] - 686s 646ms/step - loss: 0.1790 - categorical_accuracy: 0.5780
Epoch 3/80
1062/1062 [==============================] - 687s 647ms/step - loss: 0.1773 - categorical_accuracy: 0.5810
Epoch 4/80
1062/1062 [==============================] - 689s 648ms/step - loss: 0.1746 - categorical_accuracy: 0.5888
Epoch 5/80
1062/1062 [==============================] - 687s 647ms/step - loss: 0.1712 - categorical_accuracy: 0.5889
Epoch 6/80
1062/1062 [==============================] - 686s 646ms/step - loss: 0.1654 - categorical_accuracy: 0.6012
Epoch 7/80
1062/1062 [==============================] - 687s 646ms/step - loss: 0.1592 - categorical_accuracy: 0.6135
Epoch 8/80
1062/1062 [==============================] - 686s 646ms/step - loss: 0.1526 - categorical_accuracy: 0.6289
Epoch 9/80
1062/1062 [==============================] - 

In [ ]:
model.save("/content/drive/MyDrive/UNC/NCBG/mp_1b")